In [ ]:
import sys
import os

py_file_location = py_file_location = "Dirección carpeta con archivos.py AdapNet++ bimodal"
sys.path.append(os.path.abspath(py_file_location))

import argparse
import datetime
import importlib
import os
import re
import numpy as np
import tensorflow as tf
import yaml
from dataset.helper import *

In [ ]:
num_clases='Número de clases'
max_it='Número de iteraciones'
batch_size='Tamño de lote'

In [ ]:
def get_train_batch():
    filenames = ['Dirección archivo tfrecords con ambas modalidades y etiquetas']
    dataset = tf.data.TFRecordDataset(filenames)
    #parser(x,num_clases)
    dataset = dataset.map(lambda x: parser(x, num_clases))
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(100)
    dataset = dataset.prefetch(1)
    iterator = dataset.make_one_shot_iterator()
    return iterator

def get_train_data():
    iterator = get_train_batch()
    dataA, label, dataB = iterator.get_next()
    return [dataA, label, dataB], iterator

data_list, iterator = get_train_data()

In [ ]:
module = importlib.import_module('models.'+'SSMA')
model_func = getattr(module,'SSMA')
global_step = tf.Variable(0, trainable=False, name='Global_Step')
#Decay_steps=Número de iteraciones
model = model_func(num_classes=num_clases, learning_rate=0.01, decay_steps=max_it,power=0.001, global_step=global_step)
images_pl = tf.placeholder(tf.float32, [None, 384, 768, 3])
images_pl1 = tf.placeholder(tf.float32, [None, 384, 768, 3])
labels_pl = tf.placeholder(tf.float32, [None, 384, 768, num_clases])
model.build_graph(images_pl, images_pl1, labels_pl)
model.create_optimizer()
config1 = tf.ConfigProto()
config1.gpu_options.allow_growth = True
sess = tf.Session(config=config1)
sess.run(tf.global_variables_initializer())
step = 0
total_loss = 0.0
t0 = None
path1='Dirección checkpoint AdapNet++ unimodal'
path2='Dirección checkpoint AdapNet++ bimodal'
      
import_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

initialize_variables={}
#Listas con las capas de los checkpoints que se van a cargar
inspect_list1 = tf.train.list_variables(path1)
inspect_list2 = tf.train.list_variables(path2)
names=[]
names2=[]
names_no_cargar=[]
trainable_collection=tf.get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES)


for item in inspect_list1:
    names.append(item[0])
for item in inspect_list2:
    names2.append(item[0])

    #CARGAR ENCODERS 
for var in import_variables:
    if 'depth' in var.name:
        temp1=var.name.replace('depth'+'/', '')
        temp=temp1.split(':')
        if temp[0] in names:
            initialize_variables[temp[0]]=var
            
saver=tf.train.Saver(initialize_variables)         
saver.restore(sess,path1)
initialize_variables={}

for var in import_variables:
    if 'rgb' in var.name:
        temp1=var.name.replace('rgb'+'/', '')
        temp=temp1.split(':')
        if temp[0] in names:
            initialize_variables[temp[0]]=var
            
saver=tf.train.Saver(initialize_variables)    
saver.restore(sess,path1)
   #CARGAR DECODER Y SSMA
initialize_variables={}
SSMA=['conv511','conv512','conv513','conv514','conv515','conv518','conv519','conv552','conv553']
DECODER=['conv41','conv16','conv89','conv96','conv88','conv95']

for var in import_variables:
    temp1=var.name.split('/')[0]
    if temp1 in SSMA or temp1 in DECODER:
        temp=var.name.split(':')
        if temp[0] in names2:
            initialize_variables[temp[0]]=var
saver=tf.train.Saver(initialize_variables) 
saver.restore(sess,path2)
#Número de checkpoints que se quieren guardar
saver = tf.train.Saver(max_to_keep=10)

In [ ]:
perdidas=[]
while 1:
        try: 
            img, label, img1 = sess.run([data_list[0], data_list[1], data_list[2]]) 
            feed_dict = {images_pl: img, labels_pl: label, images_pl1: img1}
            loss_batch, _ = sess.run([model.loss, model.train_op],feed_dict=feed_dict)

            total_loss += loss_batch
            #Se guarda un checkpoint cada 100 iteraciones
            if (step + 1) % 100 == 0:
                saver.save(sess, os.path.join('Dirección en la que se guarda el checkpoint', 'nombre del checkpoint'), step)

            if (step + 1) % 10 == 0:
                #Cada 10 iteraciones se muestra la pérdida
                left_hours = 0

                if t0 is not None:
                    delta_t = (datetime.datetime.now() - t0).seconds
                     #left time=(numero_iteraciones-step)/10*delta_t
                    left_time = (2000 - step) / 10 * delta_t
                    left_hours = left_time/3600.0

                t0 = datetime.datetime.now()
                #media de las 10 iteraciones
                total_loss /= 10
                perdidas.append(total_loss)
                print ('%s %s] Step %s, lr = %f ' \
                  % (str(datetime.datetime.now()), str(os.getpid()), step,
                     model.lr.eval(session=sess)))
                print ('\t loss = %.4f' % (total_loss))
                print ('\t estimated time left: %.1f hours. %d/%d' % (left_hours, step,2000))
                print ('\t', 'SSMA')
                total_loss = 0.0

            step += 1
            #if step>número iteraciones
            if step > 2000:
                saver.save(sess, os.path.join('Dirección en la que se guarda el checkpoint', 'nombre del checkpoint'), step-1)
                print ('training_completed')
                break

        except tf.errors.OutOfRangeError:
            print ('Epochs in dataset repeat < max_iteration')
            break